In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('texting.csv')

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=25)
X = hv.transform(data.text)
type(X)
# Compressed Sparse Row

In [ ]:
data['hasEP'] = data['text'].str.contains('!')
data['textLen'] = data['text'].str.len()
data.head()

In [ ]:
Xhours = data.iloc[:, 1].values.reshape(data.shape[0], 1)
XhasEP = data.iloc[:, 3].values.reshape(data.shape[0], 1)
XtextLen = data.iloc[:, 4].values.reshape(data.shape[0], 1)
Xtoadd = np.concatenate((Xhours, XhasEP, XtextLen), axis=1)
print(Xtoadd)

from scipy.sparse import csr_matrix, hstack
# convert to CSR
XtoaddSparse = csr_matrix(Xtoadd)
# concatenate to BOW CSR matrix
Xfinal = hstack([X, XtoaddSparse])
X = csr_matrix(Xfinal)
type(X)

In [ ]:
# define y
y = data.iloc[:, 0].values
y.shape

In [ ]:
# create training and test sets
X_train = X[0:5]
X_test = X[5:8]
y_train = y[0:5]
y_test = y[5:8]


In [ ]:
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()